In [1]:
from cucim.skimage.feature import canny
from cucim.skimage.filters import gaussian, sobel
from cucim.skimage.color import rgb2gray, rgb2hsv
from cucim.skimage.exposure import equalize_hist


import cupy as cp

import skimage


import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import time

import warnings
from typing import Optional

warnings.filterwarnings("ignore")

# inline mode
%matplotlib inline

# read image folder path
image_folder = "/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/image/"

image_list = os.listdir(image_folder)


In [2]:
def apply_rgb2hsv(image, cuda: bool = False):

    if cuda == True:
        image = cp.asarray(image)
        image = rgb2hsv(image)
    else:
        image = skimage.color.rgb2hsv(image)

    return image


def apply_rgb2gray(image, cuda: bool = False):

    if cuda == True:
        image = cp.asarray(image)
        image = rgb2gray(image)
    else:
        image = skimage.color.rgb2gray(image)


    return image


def apply_gaussian(image, cuda: bool = False):

    if cuda == True:
        image = cp.asarray(image)
        image = gaussian(image, sigma=10)
    else:
        image = skimage.filters.gaussian(image, sigma=10)

    return image


def apply_equalize_hist(image, cuda: bool = False):

    if cuda == True:
        image = cp.asarray(image)
        image = equalize_hist(image)
    else:
        image = skimage.exposure.equalize_hist(image)

    return image


def apply_canny(image, cuda: bool = False):

    if cuda == True:
        image = rgb2gray(cp.asarray(image))
        image = canny(image)
    else:
        image = skimage.color.rgb2gray(image)
        image = skimage.feature.canny(image)

    return image


def apply_sobel(image, cuda: bool = False):

    if cuda == True:
        image = cp.asarray(image)
        image = sobel(image)
    else:
        image = skimage.filters.sobel(image)

    return image

# function list


benchmark_list = {'rgb2gray': apply_rgb2gray, 'rgb2hsv': apply_rgb2hsv, 'gaussian': apply_gaussian,
                  'equalize_hist': apply_equalize_hist, 'canny': apply_canny, 'sobel': apply_sobel}

# gpu n no gpu
gpu_available = [True, False]



In [ ]:


# benchmark function
# run benchmark for each function
for benchmark in benchmark_list.values():
    print(f"{benchmark.__name__} benchmark")
    print("="*10)
    for gpu_available in [True, False]:
        start = time.time()
        for image_name in image_list:
            image = skimage.io.imread(image_folder + image_name)
            image = benchmark(image, cuda=gpu_available)
        end = time.time()
        print(
            f"{benchmark.__name__} with {'gpu' if gpu_available else 'no_gpu'}: {end - start} seconds")
        print("="*10)


In [4]:
# single image benchmark

image = skimage.io.imread(image_folder + image_list[0])

for benchmark in benchmark_list.values():
    print(f"{benchmark.__name__} benchmark")
    print("="*10)
    for gpu_available in [True]:
        result = benchmark(image, cuda=gpu_available)
        result = result.get() if gpu_available else image
        # save image
        plt.imsave(f"/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/rapids-best-practice/image/{benchmark.__name__}_{'gpu' if gpu_available else 'no_gpu'}_cucim.png", result)


apply_rgb2gray benchmark
apply_rgb2hsv benchmark
apply_gaussian benchmark
apply_equalize_hist benchmark
apply_canny benchmark
apply_sobel benchmark


In [3]:
# single image benchmark

image = skimage.io.imread(image_folder + image_list[0])

for benchmark in benchmark_list.values():
    print(f"{benchmark.__name__} benchmark")
    print("="*10)
    for gpu_available in [False]:
        result = benchmark(image, cuda=gpu_available)
        # save image
        plt.imsave(f"/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/rapids-best-practice/image/{benchmark.__name__}_{'gpu' if gpu_available else 'no_gpu'}_cucim.png", result)


apply_rgb2gray benchmark
apply_rgb2hsv benchmark
apply_gaussian benchmark
apply_equalize_hist benchmark
apply_canny benchmark
apply_sobel benchmark


In [9]:
result = skimage.color.rgb2gray(rgb=image)

plt.imsave(f"/media/buslab/bed7bcae-c46d-4bde-874d-bdeb04d5dec9/NERO/rapids-best-practice/image/skimage_rgb2gray.png", result,cmap='gray')